In [ ]:
# default_exp seal_helper

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

# seal_helper

In [ ]:
# export
from seal import *
from typing import List

In [ ]:
# export
def print_vector(vec, print_size=4, prec=3):
    """Prints a vector with a given level of precision and print size"""
    slot_count = len(vec)
    print()
    if slot_count <= 2*print_size:
        print("    [", end="")
        for i in range(slot_count):
            print(" " + (f"%.{prec}f" % vec[i]) + ("," if (i != slot_count - 1) else " ]\n"), end="")
    else:
        print("    [", end="")
        for i in range(print_size):
            print(" " + (f"%.{prec}f" % vec[i]) + ",", end="")
        if len(vec) > 2*print_size:
            print(" ...,", end="")
        for i in range(slot_count - print_size, slot_count):
            print(" " + (f"%.{prec}f" % vec[i]) + ("," if (i != slot_count - 1) else " ]\n"), end="")
    print()
    
def ptx_value(ptx, i=0):
    """Returns the value of a Plaintext at a given position"""
    result = DoubleVector()
    encoder.decode(ptx,result)
    value = result[i]
    return value
    
def ctx_value(ctx, i=0):
    """Returns the value of a Ciphertext at a given position"""
    ptx = Plaintext()
    decryptor.decrypt(ctx, ptx)
    value = ptx_value(ptx,i)
    return value
    
def print_ctx(ctx):
    ptx = Plaintext()
    decryptor.decrypt(ctx, ptx)
    result = DoubleVector()
    encoder.decode(ptx,result)
    print_vector(result, 3, 7)
    
def print_ptx(ptx):
    result = DoubleVector()
    encoder.decode(ptx,result)
    print_vector(result, 3, 7)
    
def print_range_ctx(ctx, end=0, begin=0):
    r = range(begin,end)
    for i in r:
        print(ctx_value(ctx, i))
        
def print_range_ptx(ptx, end=0, begin=0):
    r = range(begin,end)
    for i in r:
        print(ptx_value(ptx, i))

In [ ]:
# export
def float_to_ctx(x, encoder: CKKSEncoder, encryptor: Encryptor):
    ptx = Plaintext()
    if len(x) > 1:
        x = list(x)
        x = DoubleVector(x)
    encoder.encode(x, scale, ptx)

    ctx = Ciphertext()
    encryptor.encrypt(ptx, ctx)
    
    return ctx

In [ ]:
# export
def vrep(x, n):
    k = n // len(x)
    rest = n % len(x)
    output = x * k + x[:rest]
    return output

In [ ]:
# export
def create_seal_globals(globals: dict, poly_modulus_degree: int, moduli: List[int], PRECISION_BITS: int):
    """Creates SEAL context variables and populates the globals with it."""
    parms = EncryptionParameters(scheme_type.CKKS)
    parms.set_poly_modulus_degree(poly_modulus_degree)
    parms.set_coeff_modulus(CoeffModulus.Create(
        poly_modulus_degree, moduli))

    globals["scale"] = pow(2.0, PRECISION_BITS)
    context = SEALContext.Create(parms)

    keygen = KeyGenerator(context)
    globals["public_key"] = keygen.public_key()
    globals["secret_key"] = keygen.secret_key()
    globals["relin_keys"] = keygen.relin_keys()
    globals["galois_keys"] = keygen.galois_keys()

    globals["encryptor"] = Encryptor(context, globals["public_key"])
    globals["evaluator"] = Evaluator(context)
    globals["decryptor"] = Decryptor(context, globals["secret_key"])
    globals["encoder"] = CKKSEncoder(context)
    
def append_globals_to_builtins(globals, builtins):
    """Appends the SEAL context variables to the builtins."""
    
    variables = ["public_key", "secret_key", "relin_keys", "galois_keys",
                 "encryptor", "evaluator", "decryptor", "encoder"]
    
    for var in variables:
        setattr(builtins, var, globals[var])

The examples bellow shows how one can play with SEAL.

First we initialize the SEAL context :

In [ ]:
from seal import *

poly_modulus_degree = 8192
moduli = [40,30,30,40]
PRECISION_BITS = 30

create_seal_globals(globals(), poly_modulus_degree, moduli, PRECISION_BITS)

Now we can start using the SEAL context to encrypt data and perform arithmetic on it.

In [ ]:
# First we encode x in a Plaintext
x = DoubleVector([1,2,3])

ptx = Plaintext()
encoder.encode(x, scale, ptx)

# Then we display it
print_ptx(ptx)

# Then we encrypt it
ctx = Ciphertext()
encryptor.encrypt(ptx, ctx)

print_ctx(ctx)

evaluator.add_plain_inplace(ctx, ptx)
print_ctx(ctx)


    [ 1.0000000, 1.9999999, 3.0000000, ..., -0.0000000, 0.0000000, -0.0000000 ]


    [ 0.9999997, 2.0000011, 2.9999992, ..., 0.0000006, 0.0000018, 0.0000000 ]


    [ 1.9999998, 4.0000010, 5.9999992, ..., 0.0000006, 0.0000018, -0.0000000 ]

